In [1]:
import json
import geopandas as gpd
import pandas as pd
from django.contrib.gis.geos import GEOSGeometry
from shapely.geometry import mapping
from batid.services.guess_bdg_new import Guesser, PartialRoofHandler
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

f_path = './export_rnb.gpkg'
workfile_path = "./guess.json"

In [2]:
gdf = gpd.read_file(f_path, layer="toit")

print('SRID')
srid = gdf.crs.to_epsg()
print(srid)

print("Get a sneakpeak of the data")
print(gdf.head())

print("Rows count")
print(len(gdf))



SRID
3946
Get a sneakpeak of the data
   gid  fid     id        parcelle       typetoit  hfacade  ztoitmini  \
0  NaN    1  42119  69259000CA0036  Toit non plan      5.6      217.0   
1  NaN    1  37869  69259000CE0104      Toit plan     20.2      231.8   
2  NaN    1  37885  69259000BY0082  Toit non plan      4.4      207.7   
3  NaN    1  37886  69259000BY0093      Toit plan      2.7      207.3   
4  NaN    1  37887  69259000BY0093  Toit non plan      4.4      207.7   

   ztoitmaxi  hmax  millesime  _predicate  \
0      219.8   8.4       2018  INTERSECTS   
1      231.8  20.2       2018  INTERSECTS   
2      209.1   5.8       2018  INTERSECTS   
3      207.3   2.7       2018  INTERSECTS   
4      209.1   5.8       2018  INTERSECTS   

                                            geometry  
0  POLYGON ((1844725.515 5168325.123, 1844724.025...  
1  POLYGON ((1845761.464 5167720.974, 1845759.953...  
2  POLYGON ((1845879.239 5168615.414, 1845879.364...  
3  POLYGON ((1845882.089 5168610

In [3]:
# Define function to transform geopackage rows into guess work file inputs
def to_input(row):
    
    geom_geojson = mapping(row["geometry"])
    geom = GEOSGeometry(json.dumps(geom_geojson))
    geom.srid = srid
    geom.transform(4326)
    
    
    return {
        'ext_id': row["id"],
        'polygon': json.loads(geom.json)
    }

In [4]:
# Build inputs
inputs = []
max_len = None

for idx, row in gdf.iterrows():
    inputs.append(to_input(row))
    
    if max_len and len(inputs) >= max_len:
        break

print('Sneak peak of row to inputs transformation')
print(inputs[:1])


# Add just a few guesses to check first results
guesser = Guesser()
guesser.create_work_file(inputs, workfile_path)

print(f"Guesser had {len(guesser.guesses)} guesses")



Sneak peak of row to inputs transformation
[{'ext_id': 42119, 'polygon': {'type': 'Polygon', 'coordinates': [[[4.858588636054994, 45.699804623187475], [4.8585692463174, 45.69979684270552], [4.858548765629505, 45.69980366416946], [4.858588636054994, 45.699804623187475]]]}}]
Guesser had 9862 guesses


In [5]:
import time

# Launch the guess work with specific handlers
guesser = Guesser()
guesser.handlers = [PartialRoofHandler()]

start = time.perf_counter()
guesser.guess_work_file(workfile_path)
end = time.perf_counter()


print(f"Worked on {len(guesser.guesses)} guesses")
print(f"Duration : {end - start} seconds")

Worked on 9862 guesses
Duration : 318.26511293899966 seconds


In [6]:
# Report

guesser = Guesser()
guesser.load_work_file(workfile_path)

guesser.report()

-- Report --
Number of rows: 9862
Number of match: 7720 (78.28%)

-- match_reasons : absolute --
sole_bdg_intersects_roof_enough     7371
isolated_bdg_intersects_roof         185
many_bdgs_covered_enough_by_roof     164
Name: match_reason, dtype: int64

-- match_reasons : % --
sole_bdg_intersects_roof_enough     74.741432
isolated_bdg_intersects_roof         1.875887
many_bdgs_covered_enough_by_roof     1.662949
Name: match_reason, dtype: float64


## Vérification de la présence de faux positifs

In [7]:
guesser = Guesser()
guesser.load_work_file(workfile_path)


guesser.display_reason("sole_bdg_intersects_roof_enough", 50)

KeyError: ('input_ext_id', 'match_rnb_id', 'match_reason')

Après inspection d'une quarantaine de résultats ayant une raison `one_bdg_intersects_roof_enough`, on constate l'absence de faux positifs.
On peut se demander jusqu'à combien on peut descendre le taux de recouvrement minimum (pour le moment 50%) avant d'obtenir des faux positifs.

In [ ]:
guesser = Guesser()
guesser.load_work_file(workfile_path)


guesser.display_reason("isolated_bdg_intersects_roof", 50)

Après inspection de 50 match pour raison `isolated_closest_bdg_intersects_roof` pris de façon aléatoire, on ne constate pas de faux positifs

In [ ]:
guesser = Guesser()
guesser.load_work_file(workfile_path)


guesser.display_reason("many_bdgs_covered_enough_by_roof", 20, ["input_ext_id", "match_rnb_id", "match_match_details_rnb_ids"])

## Inspection des non-match

In [ ]:
guesser = Guesser()
guesser.load_work_file(workfile_path)
guesser.display_nomatches(30)

## Vérification de cas de non-match



### 404787 -> 79QT2KSPZTYT 
- Je ne comprends pas pourquoi le résultat ne sort pas, à l'oeil je pense qu'il devrait sortir.
- Après investigation, il apparait que le bon resultat de sortait pas car nous ne retenions que les deux premiers bâtiments les plus proche, hors dans ce cas nous avions plusieurs bâtiments ayant une distance à zéro. Le bon bâtiment ne se trouvait pas parmis les deux premiers bâtiments et n'était tout simple pas retenu. Pour corriger, nous avons gardé les 20 premiers bâtiments plutot que seulement les deux premiers


### 429191 -> KAA6XQQXB91N
- il devrait sortir je pense
- après investigation : c'était un cas similaire à "404787 -> 79QT2KSPZTYT".
- NB : on avait un bug important. Si on avait un seul bâtiment intersectant assez le toit parmi la liste des plus proches, alors on retournait le premier batiment de cette liste en résultat. Le souci est que ce n'est pas forcément ce bâtiment qui était celui qui correspondait.


### 127551 -> None 
- aucun bâtiment dans le RNB
- rien à faire

### 446291 -> None 
- ne touche aucun bâtiment dans le rnb
- rien à faire

### 373773 -> RPETQK6PF2P8
- devrait sortir. Est-ce que le second bâtiment le plus proche est trop proche ? Il est à environ 6,72 mètres. Le plafond est à 10m, est-ce qu'on l'abaisse ??
- Solution : en descendant la distance minimum à 6 mètre, on obtient le bon résultat

### 441662 -> F9VWBTYH4V24 : 
- devrait sortir. Est-ce que le second bâtiment le plus proche est trop proche ? Il est environ à 6,29m !
- Solution : en descendant la distance minimum à 6 mètre, on obtient le bon résultat

### 516611 -> SSY88WB5H2ZH : 
- devrait sortir. Pourquoi il ne sort pas ?
- Solution : idem "404787 -> 79QT2KSPZTYT"

### 138328 -> WZPJ61B6Z53K 
- devrait sortir. Pourquoi il ne sort pas ?
- Solution : idem "404787 -> 79QT2KSPZTYT"


In [10]:
# Livraison CSV
import csv

guesser = Guesser()
guesser.load_work_file(workfile_path)

data = []

for guess in guesser.guesses.values():
    
    matches = None
    if guess['matches']:
        matches = guess['matches']
    
    
    data.append((guess['input']['ext_id'], matches))

with open('./rnb_lyon_toits.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(['id', 'rnb_ids'])
    # Write the data
    writer.writerows(data)
